In [1]:
import numpy as np
from src.texas_holdem_simp import Texas_Holdem_Simp

env = Texas_Holdem_Simp()

In [2]:
env.to_factor(0)

[0, 0, 0, 0, 0, 0, 0, 0]

In [3]:
env.to_tabular([0,1,0,0,2,1,2,0])

66097

# Fold

In [4]:
s = env.to_tabular([0,1,0,0,2,1,2,0])
acum = 0
for i in range(6):
    for j in range(6):
        s_ = env.to_tabular([i,j,0,0,1,1,2,0])
        p = env.T(s, 0, s_)
        acum += p
        # print(i,j, p)
print(acum)

0.9999999999999999


# Call

In [5]:
s = env.to_tabular([0,1,0,0,2,1,2,0])
acum = 0
for i in range(6):
    s_ = env.to_tabular([0,1,i,0,1,1,4,1])
    p = env.T(s, 1, s_)
    acum += p
    print(i,p)
print(acum)

0 5 0
1 5 0
2 5 0.25
3 5 0.25
4 5 0.25
5 5 0.25
1.0


# Raise

In [10]:
s = env.to_tabular([0,1,0,0,2,1,2,0])
acum = 0
for i in range(6):
    s_ = env.to_tabular([0,1,i,0,0,2,6,1])
    p = env.T(s, 2, s_)
    acum += p
    print(i,p)
print(acum)

0 5 0
1 5 0
2 5 0.25
3 5 0.25
4 5 0.25
5 5 0.25
1.0


# Allin

In [12]:
s = env.to_tabular([0,1,0,0,2,1,2,0])
acum = 0
for i in range(7):
    s_ = env.to_tabular([0,0,0,0,i,0,0,0])
    p = env.T(s, 3, s_)
    acum += p
    print(i,p)
print(acum)

0 5 1
1 5 0
2 5 0
3 5 0
4 5 0
5 5 0
6 5 0
1


In [9]:
s = env.to_tabular([0,1,0,0,2,1,2,0])
s_ = env.to_tabular([0,1,2,0,0,2,6,1])
print(s)
print(s_)
env.T(s, 2, s_)

66097
372025


0.25

In [3]:
env.R(None, None, 3)

-1